In [2]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
from tabulate import tabulate

#webscrapping
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
data = pd.DataFrame(df[0])
data.head()

#naming the columns 
x = data.rename(columns = data.iloc[0,])
x.head()
y = x.drop(x.index[0])
y.head()

#resetting the index

df = y.reset_index(drop=True)
df.head()

#ignoring "not assigned" borough

df_updated = df[~df.Borough.str.contains("Not assigned")]
df_updated.head()

#Making Neighbourhood same as Borough conditioning Neighbourhood = Not Assigned
df_updated['Neighbourhood'] = df_updated.apply(lambda x: x["Borough"] if x["Neighbourhood"] == "Not assigned" else x['Neighbourhood'], axis = 1)


#Grouping the dataframe based on Postcode and merging the same Neighbourhood
df_final = df_updated.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
df_final

df_final.shape

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(103, 3)

# Merging of Longitude Latitude 

In [3]:
#read csv
geocode = pd.read_csv("C:/Users/Dell/Geospatial_Coordinates.csv")

#rename 
geocode = geocode.rename(columns = {'Postal Code' : 'Postcode'})
geocode.head()



,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


MERGING

In [4]:
final_df = pd.merge(df_final, geocode[['Postcode','Latitude','Longitude']], on = 'Postcode')
final_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
